In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('nba_games.csv', index_col=0)

In [3]:
df

,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,fga_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
season_id,,,,,,,,,,,,,,,,,,,,,
21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,2,0,Regular Season
21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,59.0,...,NaN,NaN,NaN,NaN,NaN,25.0,51.0,-5,0,Regular Season
21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,53.0,-6,0,Regular Season
21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,22.0,47.0,-16,0,Regular Season
21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,50.0,17,0,Regular Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42022,1610612748,MIA,Miami Heat,42200403,2023-06-07 00:00:00,MIA vs. DEN,L,240,34.0,92.0,...,58.0,28.0,3.0,5.0,14.0,18.0,109.0,15,1,Playoffs
42022,1610612748,MIA,Miami Heat,42200404,2023-06-09 00:00:00,MIA vs. DEN,L,240,35.0,78.0,...,34.0,26.0,11.0,7.0,8.0,18.0,108.0,13,1,Playoffs
42022,1610612743,DEN,Denver Nuggets,42200405,2023-06-12 00:00:00,DEN vs. MIA,W,240,38.0,84.0,...,44.0,18.0,9.0,7.0,8.0,21.0,89.0,-5,1,Playoffs


In [4]:
df = df.sort_values('date')

KeyError: 'date'

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
del df['mp_opp.1']
del df['mp.1']
del df['index_opp']

In [ ]:
df

In [ ]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

df = df.groupby('team', group_keys=False).apply(add_target)

In [ ]:
df[df['team'] == 'DEN']

In [ ]:
df['target'][pd.isnull(df['target'])] = 2

In [ ]:
df['target'] = df['target'].astype(int, errors='ignore')

In [ ]:
df['won'].value_counts()

In [ ]:
df['target'].value_counts()

In [ ]:
nulls = pd.isnull(df)

In [ ]:
nulls = nulls.sum()

In [ ]:
nulls

In [ ]:
nulls = nulls[nulls > 0]

In [ ]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [ ]:
valid_columns

In [ ]:
 df = df[valid_columns].copy()

In [ ]:
df

Até agora, nós limpamos e preparamos os dados para alimentar o modelo. Apartir de agora, começarei a treinar o modelo

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction='backward', cv=split)

In [ ]:
removed_columns = ['season', 'date', 'won', 'target', 'team', 'team_opp']

In [ ]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [ ]:
selected_columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [ ]:
df

In [ ]:
sfs.fit(df[selected_columns], df['target'])

In [ ]:
predictors = list(selected_columns[sfs.get_support()])

In [ ]:
predictors

In [ ]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []

    seasons = sorted(data['season'].unique())

    for i in range(start, len(seasons), step):
        season =  seasons[i]

        train = data[data['season'] < season]
        test = data[data['season'] == season]

        model.fit(train[predictors], train['target'])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)

        combined = pd.concat([test['target'], preds], axis=1)
        combined.columns = ['actual', 'prediction']

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [ ]:
predictions = backtest(df, rr, predictors) 

In [ ]:
predictions

In [ ]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions['actual'] != 2]
accuracy_score(predictions['actual'], predictions['prediction'])

Inicialmente o modelo não teve uma boa acurácia, 54.85%, os próximos blocos de código serão dedicados a melhorá-la 

In [ ]:
df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

In [ ]:
df

In [ ]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

# Certifique-se de incluir apenas colunas numéricas no cálculo
numeric_columns = df_rolling.select_dtypes(include='number').columns

# Atualize a função para trabalhar apenas com colunas numéricas
def find_team_averages(team):
    # Aplicar rolling somente nas colunas numéricas
    rolling = team[numeric_columns].rolling(10).mean()
    return rolling

# Agrupe e aplique a função corrigida
df_rolling = df_rolling.groupby(['team', 'season'], group_keys=False).apply(find_team_averages)


In [ ]:
df_rolling

In [ ]:
rolling_cols = [f'{col}_10' for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [ ]:
df

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name): 
    return df.groupby('team', group_keys=False).apply(lambda x: shift_col(x, col_name))

df['home_next'] = add_col(df, 'home')
df['team_opp_next'] = add_col(df, 'team_opp')
df['date_next'] = add_col(df, 'date')

In [ ]:
df = df.copy()

In [ ]:
full = df.merge(
    df[rolling_cols + ["team_opp_next", "date_next", "team"]], 
    left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

In [ ]:
full

In [ ]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

In [ ]:
 removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [ ]:
removed_columns

In [ ]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
sfs.fit(full[selected_columns], full["target"])

In [ ]:
predictors = list(selected_columns[sfs.get_support()])
predictors

In [ ]:
predictions = backtest(full, rr, predictors)

In [ ]:
accuracy_score(predictions["actual"], predictions["prediction"])